In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
from jax import config

config.update("jax_enable_x64", True)
# config.update("jax_platform_name", "cpu")

import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".8"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import matplotlib as mpl

import jaxley as jx

In [6]:
rec_id = 'd1'  # Can pick any here.

import os
home_directory = os.path.expanduser("~")
data_dir = f'{home_directory}/GitRepos/jaxley_rgc/extracted_data'
setup_file = [f for f in os.listdir(data_dir) if f.endswith('.pkl') and f.startswith('cell')]
print(setup_file)

all_dfs = pd.read_pickle(f'{data_dir}/{setup_file[0]}')
recording_id = all_dfs.loc[0,'recording_id']
morph_file = [f for f in os.listdir(f'{data_dir}/morphology/{recording_id}') if f.endswith('.swc')]



assert os.path.isdir(data_dir)

['cell_2020-07-08.pkl']


In [4]:
cell = jx.read_swc(f"{data_dir}/morphology/{recording_id}/{morph_file[0]}", nseg=4, max_branch_len=300.0, min_radius=5.0)

/gpfs01/euler/User/ssuhai/GitRepos/jaxley/jaxley/utils/misc_utils.py:97: UserWarning: nseg is deprecated and will be removed in version 0.6.0.
  warnings.warn(msg + self._amend_msg)
E0117 16:46:59.356877   22956 hlo_lexer.cc:443] Failed to parse int literal: 58800360702711838510


In [7]:
bc_file = f"{data_dir}/off_bc_output_rec_id_{recording_id}_{rec_id}.pkl"
bc_output_df = pd.read_pickle(bc_file)

In [8]:
stim = bc_output_df
stim = stim[stim["rec_id"] == rec_id]

In [9]:
def compute_jaxley_stim_locations(x, y):
    """For a given (x,y) location, return all branch and compartment inds within a specified distance."""
    min_dists = []
    min_comps = []
    branch_inds_in_pixel = []
    comps_in_pixel = []
    min_dist_of_branch_in_pixel = []

    for i, xyzr in enumerate(cell.xyzr):
        dists = np.sqrt((x - xyzr[:, 0])**2 + (y - xyzr[:, 1])**2)
        is_in_reach = np.min(dists) < 20  # 20 um

        if is_in_reach:
            branch_inds_in_pixel.append(i)
            min_dist_of_branch_in_pixel.append(np.min(dists))
            
            argmin_dist = np.argmin(dists)
            if len(dists) > 1:
                comp = argmin_dist / (len(dists) - 1)
            else:
                comp = 0.5
            comps_in_pixel.append(comp)
            
    return branch_inds_in_pixel, comps_in_pixel, min_dist_of_branch_in_pixel

In [10]:
bc_loc_x = stim["x_loc"].to_numpy()
bc_loc_y = stim["y_loc"].to_numpy()
bc_ids = stim["bc_id"].to_numpy()

In [12]:
bcs_which_stimulate = 0

branch_inds_for_every_bc = []
comp_inds_for_every_bc = []
mind_dists_of_branches_for_every_bc = []
bc_ids_per_stim = []

for x, y, id in zip(bc_loc_x, bc_loc_y, bc_ids):
    branches, comps, min_dist_of_branch_in_pixel = compute_jaxley_stim_locations(x, y)
    branch_inds_for_every_bc += branches
    comp_inds_for_every_bc += comps
    mind_dists_of_branches_for_every_bc += min_dist_of_branch_in_pixel
    bc_ids_per_stim += [id] * len(branches)

In [ ]:
#TODO:again this is confusing
cell_id =recording_id

In [15]:
stim_df = pd.DataFrame().from_dict(
    {
        "cell_id": cell_id, 
        "bc_id": bc_ids_per_stim, 
        "branch_ind": branch_inds_for_every_bc, 
        "comp": comp_inds_for_every_bc, 
        "dist_from_bc": mind_dists_of_branches_for_every_bc
    }
)

In [18]:
stim_df

,cell_id,bc_id,branch_ind,comp,dist_from_bc,num_synapses_of_bc
0,2020-07-08,17,19,0.964286,17.850997,1
1,2020-07-08,26,19,0.765306,1.225495,1
2,2020-07-08,29,23,0.982955,1.113831,1
3,2020-07-08,35,18,0.970149,1.795016,1
4,2020-07-08,36,17,0.902439,7.369344,2
...,...,...,...,...,...,...
139,2020-07-08,107,33,0.156863,17.907813,2
140,2020-07-08,107,48,0.529730,17.576014,2
141,2020-07-08,108,32,0.392638,3.941906,1
142,2020-07-08,109,43,0.539877,7.322142,2


In [17]:
stim_df["num_synapses_of_bc"] = stim_df.groupby("bc_id").bc_id.transform(len)

In [19]:
stim_df.to_pickle(f"{data_dir}/stimuli_meta_{cell_id}.pkl")